In [38]:
import numpy as np
import pandas as pd
import nltk
import sklearn

In [39]:
df = pd.read_csv('./Train_Data.csv')
df.head()

,text,author,controversiality,parent_text,parent_score,parent_votes,parent_author,parent_controversiality,Score
0,i must be retarded i thought it meant con lawl...,['calantus'],0,"It's quite unfair to call Hillary Clinton a ""c...",245,245,Whisper,0,-8
1,DOWNMODDED FOR IRRELEVANCE? ISN'T THAT HOW THI...,['Shadowrose'],0,upmodded for awesome kindness,32,32,b3mus3d,0,-16
2,"THAT WAS SUPPOSED TO MEAN "" BY A PLACE WHERE P...",['NExusRush'],0,"What the hell does ""because its by a golf cour...",12,12,mr_jellyneck,0,-7
3,I THOUGHT EVERYONE DID; ITS FUCKING DELICIOUS :\,['R0N_SWANS0N'],0,NICE TRY JENNIFER! I KNOW IT'S YOU AND I KNOW...,117,117,ometzo,0,67
4,"Great work, Zhesbe! I'd give you a raise but y...",['reddums'],0,"""HEY BOSS COME LOOK AT WHAT I DID!""",1933,1933,Zhesbe,0,1348


In [40]:
df.shape

(4999, 9)

In [41]:
df.isnull().sum()

text                       0
author                     0
controversiality           0
parent_text                0
parent_score               0
parent_votes               0
parent_author              0
parent_controversiality    0
Score                      0
dtype: int64

In [42]:
df.nunique()

text                       4992
author                     4317
controversiality              2
parent_text                4992
parent_score               1001
parent_votes               1001
parent_author              4448
parent_controversiality       2
Score                       572
dtype: int64

In [43]:
print(df['parent_score'].equals(df['parent_votes']))

True


In [44]:
df.controversiality.value_counts()

0    4996
1       3
Name: controversiality, dtype: int64

In [45]:
df.parent_controversiality.value_counts()

0    4993
1       6
Name: parent_controversiality, dtype: int64

In [46]:
df = df.drop(columns=['author','parent_author','parent_votes'])

##### Remove Punctuation

In [47]:
cat_cols = ['text','parent_text']

for col in cat_cols:
    df[col] = df[col].str.lower()
    df[col] = df[col].str.strip()

In [48]:
df.head()

,text,controversiality,parent_text,parent_score,parent_controversiality,Score
0,i must be retarded i thought it meant con lawl...,0,"it's quite unfair to call hillary clinton a ""c...",245,0,-8
1,downmodded for irrelevance? isn't that how thi...,0,upmodded for awesome kindness,32,0,-16
2,"that was supposed to mean "" by a place where p...",0,"what the hell does ""because its by a golf cour...",12,0,-7
3,i thought everyone did; its fucking delicious :\,0,nice try jennifer! i know it's you and i know...,117,0,67
4,"great work, zhesbe! i'd give you a raise but y...",0,"""hey boss come look at what i did!""",1933,0,1348


In [49]:
import string

for col in cat_cols:
    df[col] = df[col].apply(lambda x : ''.join([char for char in x if char not in string.punctuation]))
df.head()

,text,controversiality,parent_text,parent_score,parent_controversiality,Score
0,i must be retarded i thought it meant con lawl...,0,its quite unfair to call hillary clinton a cun...,245,0,-8
1,downmodded for irrelevance isnt that how this ...,0,upmodded for awesome kindness,32,0,-16
2,that was supposed to mean by a place where pe...,0,what the hell does because its by a golf cours...,12,0,-7
3,i thought everyone did its fucking delicious,0,nice try jennifer i know its you and i know y...,117,0,67
4,great work zhesbe id give you a raise but you ...,0,hey boss come look at what i did,1933,0,1348


##### Tokenization

In [50]:
import re
def tokenize(text):
    tokens = re.split('\W',text)
    words = [w for w in tokens if w.isalpha()]
    return words

In [51]:
for col in cat_cols:
    df[col] = df[col].apply(lambda x: tokenize(x))

In [52]:
df.head()

,text,controversiality,parent_text,parent_score,parent_controversiality,Score
0,"[i, must, be, retarded, i, thought, it, meant,...",0,"[its, quite, unfair, to, call, hillary, clinto...",245,0,-8
1,"[downmodded, for, irrelevance, isnt, that, how...",0,"[upmodded, for, awesome, kindness]",32,0,-16
2,"[that, was, supposed, to, mean, by, a, place, ...",0,"[what, the, hell, does, because, its, by, a, g...",12,0,-7
3,"[i, thought, everyone, did, its, fucking, deli...",0,"[nice, try, jennifer, i, know, its, you, and, ...",117,0,67
4,"[great, work, zhesbe, id, give, you, a, raise,...",0,"[hey, boss, come, look, at, what, i, did]",1933,0,1348


##### Remove Stopwords

In [53]:
import requests
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines()) 

In [54]:
def remove_stopwords(text_token):
    without_stopwords = [x for x in text_token if x not in stopwords]
    return without_stopwords

##### Vectorize

In [55]:
for col in cat_cols:
    df[col] = df[col].apply(lambda x: ' '.join(x))
df.head()

,text,controversiality,parent_text,parent_score,parent_controversiality,Score
0,i must be retarded i thought it meant con lawl...,0,its quite unfair to call hillary clinton a cun...,245,0,-8
1,downmodded for irrelevance isnt that how this ...,0,upmodded for awesome kindness,32,0,-16
2,that was supposed to mean by a place where peo...,0,what the hell does because its by a golf cours...,12,0,-7
3,i thought everyone did its fucking delicious,0,nice try jennifer i know its you and i know yo...,117,0,67
4,great work zhesbe id give you a raise but you ...,0,hey boss come look at what i did,1933,0,1348


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_features=50,min_df=1,max_df=0.7)
text_tfidf = tfidf_vect.fit_transform(df['text']).toarray()
text_tfidf = pd.DataFrame(text_tfidf,columns=tfidf_vect.get_feature_names())

In [58]:
text_tfidf.shape

(4999, 50)

In [60]:
num_cols = df[['controversiality','parent_score','parent_controversiality']]
x = pd.concat([text_tfidf,num_cols],axis=1)
y = df['Score']

In [61]:
x.shape

(4999, 53)

##### ML Model

In [62]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=34)

In [63]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
model = lr.fit(X_train,y_train)
y_pred= model.predict(X_test)

In [64]:
from sklearn.metrics import mean_squared_error
print('root mean squared error:',np.sqrt(mean_squared_error(y_test,y_pred)))

root mean squared error: 162.7575809249547


In [68]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':np.arange(1,100)}
knn = KNeighborsRegressor()
knn_cv = GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [69]:
knn_cv.best_params_

{'n_neighbors': 98}

In [70]:
knn = KNeighborsRegressor(n_neighbors=98)
knn.fit(X_train,y_train)
y_pred2 = knn.predict(X_test)
print('root_mean_squared_error:',np.sqrt(mean_squared_error(y_test,y_pred2)))

root_mean_squared_error: 157.8923881894263


In [74]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
param_grid = {'n_estimators':np.arange(1,50),'max_depth':np.arange(1,50)}
rf = RandomForestRegressor()
rf_cv = RandomizedSearchCV(rf,param_grid,cv=5)
rf_cv.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                                        'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [75]:
rf_cv.best_params_

{'n_estimators': 27, 'max_depth': 8}

In [76]:
rf = RandomForestRegressor(n_estimators=27,max_depth=8,max_features='auto')
rf.fit(X_train,y_train)
y_pred3 = rf.predict(X_test)

print('root mean squared error:',np.sqrt(mean_squared_error(y_test,y_pred3)))

root mean squared error: 162.57561362146066
